# Persiapan Librari dan Dataset

In [ ]:
import cv2
import numpy as np
import os
import joblib
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mahotas as mt
from skimage.feature import graycomatrix, graycoprops
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

print("Semua librari berhasil diimpor.")

In [ ]:
dataset_path = r"D:\dataset\data sekunder"

In [ ]:
categories = sorted([d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))])
print("Kategori ditemukan:", categories)

# Preprocessing dan Segmentasi

In [ ]:
def preprocess_segment(image_path):

    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Gagal membaca gambar dari path: {image_path}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized_image = cv2.resize(image, (512, 512))
    brightness = 5
    adjusted_image = cv2.convertScaleAbs(resized_image, alpha=1, beta=brightness)
    blurred_image = cv2.GaussianBlur(adjusted_image, (7, 7), 0)
    hsv = cv2.cvtColor(blurred_image, cv2.COLOR_RGB2HSV)

    lower_green = np.array([35, 30, 30])
    upper_green = np.array([90, 255, 255])
    lower_brown = np.array([10, 30, 10])
    upper_brown = np.array([80, 255, 255])
    lower_yellow = np.array([10, 50, 40])
    upper_yellow = np.array([40, 255, 255])

    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    mask_brown = cv2.inRange(hsv, lower_brown, upper_brown)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    mask_combined = cv2.bitwise_or(mask_green, cv2.bitwise_or(mask_brown, mask_yellow))

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask_cleaned = cv2.morphologyEx(mask_combined, cv2.MORPH_OPEN, kernel, iterations=2)
    mask_cleaned = cv2.morphologyEx(mask_cleaned, cv2.MORPH_CLOSE, kernel, iterations=2)

    contours, _ = cv2.findContours(mask_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cleaned_mask = np.zeros_like(mask_cleaned)
    min_contour_area = 2500
    for contour in contours:
        area = cv2.contourArea(contour)
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = w / h
        if area > min_contour_area and 0.1 < aspect_ratio < 3.0:
            cv2.drawContours(cleaned_mask, [contour], -1, 255, thickness=cv2.FILLED)

    segmented = cv2.bitwise_and(resized_image, resized_image, mask=cleaned_mask)

    return segmented

# Ekstraksi Fitur

In [ ]:
from math import radians
bins = 8

def extract_hsv_histogram(segmented):
    hsv = cv2.cvtColor(segmented, cv2.COLOR_RGB2HSV)
    hist_h = cv2.calcHist([hsv], [0], None, [bins], [0, 180])
    hist_s = cv2.calcHist([hsv], [1], None, [bins], [0, 256])
    hist_v = cv2.calcHist([hsv], [2], None, [bins], [0, 256])
    hist_features = np.concatenate([hist_h.flatten(), hist_s.flatten(), hist_v.flatten()])
    return hist_features / np.sum(hist_features)

def extract_glcm_features(segmented):
    gray = cv2.cvtColor(segmented, cv2.COLOR_RGB2GRAY)
    angles = [radians(0), radians(45), radians(90), radians(135)]
    glcm = graycomatrix(gray, distances=[1], angles=angles, levels=256, symmetric=True, normed=True)

    contrast = np.mean(graycoprops(glcm, 'contrast'))
    correlation = np.mean(graycoprops(glcm, 'correlation'))
    energy = np.mean(graycoprops(glcm, 'energy'))
    homogeneity = np.mean(graycoprops(glcm, 'homogeneity'))

    return np.array([contrast, correlation, energy, homogeneity])

In [ ]:
X, y = [], []

for category in categories:
    category_path = os.path.join(dataset_path, category)

    for image_name in os.listdir(category_path):
        image_path = os.path.join(category_path, image_name)
        
        segmented = preprocess_segment(image_path)
        if segmented is not None:
            hsv_features = extract_hsv_histogram(segmented)
            glcm_features = extract_glcm_features(segmented)

            features = np.concatenate((hsv_features, glcm_features))
            X.append(features)
            y.append(category)

X = np.array(X)
y = np.array(y)

print(f"Dataset siap dengan {X.shape[0]} sampel dan {X.shape[1]} fitur.")

np.savez("static/feature extraction/dataset_tomat_features.npz", X=X, y=y)

Melihat Label

In [ ]:
data = np.load("static/Feature Extraction/dataset_tomat_features.npz", allow_pickle=True)
X, y = data["X"], data["y"]

print(f"Dataset dimuat dengan {X.shape[0]} sampel dan {X.shape[1]} fitur.")

In [ ]:
# Tampilkan semua label unik
unique_labels = np.unique(y)
print("Label unik dalam dataset:", unique_labels)

# Pelatihan Model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [ 'scale', 'auto'],
    'kernel': ['linear', 'poly', 'rbf'],
}

In [ ]:
svm = SVC()
grid_search = GridSearchCV(svm, param_grid,cv=5,scoring='accuracy', n_jobs=-1, return_train_score=True)
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)

In [ ]:
best_svm = grid_search.best_estimator_

In [ ]:
y_pred = best_svm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi Model SVM: {accuracy * 100:.2f}%")

In [ ]:
print("Decision function shape:", best_svm.decision_function_shape)

# Simpan dan Load Model

In [ ]:
joblib.dump(best_svm, "model/svm_model.pkl")
joblib.dump(scaler, "model/scaler.pkl")

print("\nModel SVM berhasil disimpan sebagai 'svm_model.pkl' dan 'scaler.pkl'.")

In [ ]:
svm_model = joblib.load("model/svm_model.pkl")
scaler = joblib.load("model/scaler.pkl")

def predict_tomato_disease(features):
    features = np.array(features).reshape(1, -1)
    features = scaler.transform(features)
    prediction = svm_model.predict(features)[0]
    return prediction

# Evaluasi dan Visualisasi Model

Visualisasi Confusion Matrix

In [ ]:
y_pred = svm_model.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="cividis", xticklabels=categories, yticklabels=categories)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
for i, class_label in enumerate(categories):
    TP = conf_matrix[i, i]
    FN = conf_matrix[i, :].sum() - TP
    FP = conf_matrix[:, i].sum() - TP
    TN = conf_matrix.sum() - (TP + FP + FN)

    print(f"Class {class_label}: TP={TP}, FN={FN}, FP={FP}, TN={TN}")

Akurasi Data Training & Testing

In [ ]:
train_accuracy = accuracy_score(y_train, svm_model.predict(X_train))
test_accuracy = accuracy_score(y_test, y_pred)

print(f"Akurasi Training: {train_accuracy:.4f}")
print(f"Akurasi Testing: {test_accuracy:.4f}")

In [ ]:
train_accuracy = accuracy_score(y_train, svm_model.predict(X_train))
test_accuracy = accuracy_score(y_test, y_pred)
labels = ['Training', 'Testing']
accuracies = [train_accuracy, test_accuracy]
sns.set(style='whitegrid')
plt.figure(figsize=(5, 5))
palette = sns.color_palette("pastel")
ax = sns.barplot(x=labels, y=accuracies, palette=palette)

for i, acc in enumerate(accuracies):
    ax.text(i, acc + 0.02, f'{acc:.4f}', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.title('Perbandingan Akurasi Model SVM', fontsize=16, fontweight='bold')
plt.ylabel('Akurasi', fontsize=12)
plt.ylim(0, 1.05)
sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.show()

Classification Report

In [ ]:
print("Classification Report Data 80:20")
print(classification_report(y_test, y_pred, target_names=categories))

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(svm_model, X_train, y_train, cv=5)
print(scores.mean(), scores.std())

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(svm_model, X_train, y_train, cv=5)

print("Akurasi per fold:", scores)              # semua skor fold
print("Rata-rata akurasi:", scores.mean())      # rata-rata
print("Standar deviasi:", scores.std())         # deviasi standar

# Prediksi Gambar Baru

In [ ]:
folder_primer = r"D:\dataset\data primer"

In [ ]:
hasil_prediksi = []

for image_name in os.listdir(folder_primer):
    image_path = os.path.join(folder_primer, image_name)
    
    if not image_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    try:
        segmented = preprocess_segment(image_path)
        if segmented is not None:
            hsv_features = extract_hsv_histogram(segmented)
            glcm_features = extract_glcm_features(segmented)
            features = np.concatenate((hsv_features, glcm_features))
            predicted_label = predict_tomato_disease(features)

            hasil_prediksi.append([image_name, predicted_label])
    except Exception as e:
        hasil_prediksi.append([image_name, f"Error: {e}"])

# Simpan ke CSV
df_prediksi = pd.DataFrame(hasil_prediksi, columns=["Nama Gambar", "Prediksi"])
df_prediksi.to_csv("static/primer prediction/hasil_prediksi_data_primer.csv", index=False)
print("\nHasil prediksi disimpan ke 'hasil_prediksi_data_primer.csv'")


# Simpan File CSV

In [ ]:
data = np.column_stack((X, y))
hsv_columns = [f'H_bin_{i}' for i in range(bins)] + \
              [f'S_bin_{i}' for i in range(bins)] + \
              [f'V_bin_{i}' for i in range(bins)]
glcm_columns = ['contrast', 'correlation', 'energy', 'homogeneity']
columns = hsv_columns + glcm_columns + ['label']
df = pd.DataFrame(data, columns=columns)
df.to_csv("static/feature extraction/fitur-daun.csv", index=False)

print("Dataset berhasil disimpan dalam format CSV dengan", len(columns), "kolom.")

# Baca Dataset Hasil Ekstraksi Fitur

In [ ]:
data = pd.read_csv('static/feature extraction/fitur-daun.csv')

In [ ]:
filtered_data_busuk_daun = data[data['label'] == 'busuk daun'].head(2)
filtered_data_jamur_daun = data[data['label'] == 'jamur daun'].head(2)
filtered_data_sehat = data[data['label'] == 'sehat'].head(2)
filtered_data_septoria = data[data['label'] == 'septoria'].head(2)
filtered_data = pd.concat([filtered_data_busuk_daun, filtered_data_jamur_daun, filtered_data_sehat, filtered_data_septoria])
pd.set_option('display.max_columns', 29)
filtered_data